<a href="https://colab.research.google.com/github/AminaTkh/AlanTuringInstitute_Bias_in_AI/blob/main/Forms-and-Quantification-of-Bias-in-AI/Practice_3_Evaluate_fairness_metrics_with_holisticai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Milestone 3: Sources, Forms, and Quantification of Bias and Discrimination in Supervised Learning**
## **PRACTICE NOTEBOOK 3 - Evaluate model bias using holisticai library**


In this part of the course, we will look for bias using a practical example. A  company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they're not selected or 1 if they are.

There are 4 practice notebooks in total (current one in red):
1. Explore given data to: detect potential bias early & check for proxies
2. Evaluate model bias "manually"
3. <font color='red'> **Evaluate model bias using holisticai library**</font>
4. Example code to get confidence intervals for a metric (nothing to do)

Instructions to complete in each parts are in bold. Intermediate results are given so one can continue the exercise.

This is notebook number 3. In the previous notebook, we measured bias manually. There exist a number of python library facilitating calculations for us. In this part, we will replicate the results found in the previous section using holisticai. In this notebook, we will:
- Import data and useful modules
- Install and import holisticai
- Learn how to calculate fairness metrics using holisticai

We evaluate here the same metrics as in the previous notebook. You should get the same results as what you calculated "manually" in notebook 2. Here we only work with the "Black" unprivileged group.



## **0 - Import modules, load data and useful functions**

In [1]:
#imports
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Load data

from sklearn.datasets import fetch_openml
bunch = fetch_openml(data_id=44270)
raw_data = bunch['frame']
raw_data

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


,Label,Gender,Ethnicity,0,1,2,3,4,5,6,...,40,41,42,43,44,45,46,47,48,49
0,1,Female,Black,-0.178832,0.147077,0.775331,-0.427889,0.640818,-0.610427,-1.023371,...,0.513206,-0.042387,-0.355889,-0.465837,-2.832634,0.917297,-0.241052,-2.122105,0.253170,0.164617
1,1,Male,Hispanic,0.092276,0.122023,0.482935,-0.232131,-1.939064,-1.140216,-0.833250,...,-1.203151,0.750136,-1.417751,1.254152,0.631731,1.665469,-0.388293,-0.804782,-0.227182,0.412375
2,0,Female,Hispanic,-1.703377,-0.962149,-0.785495,-0.633902,-0.334718,-1.555958,0.825006,...,0.249654,0.368800,0.079523,-0.932425,-0.693293,-0.114197,-1.252067,0.834270,-0.463270,0.559294
3,0,None,None,-1.229715,-1.342997,-0.093382,0.136900,1.195076,-0.582687,0.052355,...,0.667730,0.595494,0.454502,-0.366884,-0.399758,1.113102,0.126707,0.474569,-0.158103,1.197710
4,1,Male,Hispanic,-0.363013,1.264307,1.667603,0.903941,-0.062840,0.680886,0.389930,...,-0.494161,0.784050,-0.311236,2.447118,1.127650,0.086733,-0.381553,0.209684,0.197809,-0.879914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,Female,Hispanic,-0.057075,1.791513,-1.065756,-0.783341,-0.559215,1.042646,-1.154058,...,-0.335082,0.709753,0.021583,1.718576,1.171804,0.430075,3.340726,1.349216,1.481516,0.070563
9996,1,Male,Black,0.582066,0.086788,0.167259,-1.672798,1.537135,-1.113315,0.222907,...,-0.562594,0.150314,-0.072920,-1.841719,-0.807065,-0.793955,-1.098300,-1.474154,-0.828826,-0.891166
9997,0,Male,White,-1.355098,-0.321228,-0.204290,0.498632,1.634130,0.847070,-0.552140,...,-1.204719,0.688433,-1.781911,0.275032,0.690859,0.666878,0.644440,0.127891,1.277781,-0.744428
9998,1,Male,White,-0.526557,2.174463,-0.979082,-0.681536,-0.145515,1.703135,0.947010,...,-0.487367,2.043764,-1.570147,0.861712,-0.939181,0.090775,-1.153183,-1.362903,-1.424866,-0.374579


In [3]:
# remove all nans --> we use the variable "data" in the rest of this notebook
data = raw_data.dropna()

In [4]:
def plot_cm(y_true,y_pred,labels = [1,0],display_labels = [1,0], ax = None):
  cm = confusion_matrix(y_true,y_pred,labels = labels)
  if ax is None:
    fig, ax = plt.subplots()
  else:
    fig = ax.figure
  sns.heatmap(cm, annot=True, ax = ax, cmap='viridis',fmt='g')

  ax.set(xticklabels=display_labels,
          yticklabels=display_labels,
          ylabel="True label",
          xlabel="Predicted label")
  return cm
def split_data_from_df(data):
  y = data['Label'].values
  # g = data['Gender'].values
  # e = data['Ethnicity'].values
  X = data[np.arange(50).astype(str)].values
  filter_col = ['Ethnicity','Gender'] + [col for col in data if str(col).startswith('Ethnicity_')] + [col for col in data if str(col).startswith('Gender_')]
  dem = data[filter_col].copy()
  return X,y,dem
def encode(df):
  g_enc = LabelEncoder()
  e_enc = LabelEncoder()
  df['Gender'] = g_enc.fit_transform(df['Gender'])
  df['Ethnicity'] = e_enc.fit_transform(df['Ethnicity'])
  return df, g_enc,e_enc
def resample_equal(df,cat):
  df['uid'] = df[cat] + df['Label'].astype(str)
  enc = LabelEncoder()
  df['uid'] = enc.fit_transform(df['uid'])
  # Resample
  uid = df['uid'].values
  res = imblearn.over_sampling.RandomOverSampler(random_state=6)
  df_res,euid = res.fit_resample(df,uid)
  df_res = pd.DataFrame(df_res,columns = df.columns)
  df_res = df_res.sample(frac=1).reset_index(drop=True)
  df_res['Label'] = df_res['Label'].astype(float)
  return df_res

## **1. Install and import holisticai library**

In [5]:
# !pip install holisticai
!pip3 install holisticai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.4/241.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 10.5 MB/s eta 0:00:00


In [6]:
# import some bias metrics
from holisticai.bias.metrics import statistical_parity
from holisticai.bias.metrics import disparate_impact
from holisticai.bias.metrics import equal_opportunity_diff

## **2. Train the model and get the predictions for the test set**

- **Train the model and get the predictions for the test set.**

In [7]:
# split into train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=4)
# get X,y,demographics for each
X_train,y_train,dem_train = split_data_from_df(data_train)
X_test,y_test,dem_test = split_data_from_df(data_test)
# define model and train
model = RidgeClassifier(random_state=42)
model.fit(X_train,y_train)
# evaluate on test
y_pred_test = model.predict(X_test)
acc = accuracy_score(y_test,y_pred_test)
print("Accuracy is %.2f"%acc)
# add predictions to data_test for simplicity of analysis
data_test = data_test.copy()
data_test['Pred'] = y_pred_test

Accuracy is 0.69


## **4. Compute fairness metrics**

Now we will need to choose what privileged group and unprivileged group we want to focus on. We will demonstrate for *Black* vs *White*. We define our privileged group and unprivileged group as such. Link to [holisticai](https://holisticai.readthedocs.io/en/latest/metrics.html#binary-classification) documentation.

In [10]:
# set up vectors
group_a = data_test.Ethnicity=='Black'
group_b = data_test.Ethnicity=='White'
y_true = y_test
y_pred = y_pred_test

# compute fairness metrics
results = pd.DataFrame(columns = ['Metric', 'Value'])
results.loc[len(results)] = ['Statistical Parity', statistical_parity(group_a, group_b, y_pred)]
results.loc[len(results)] = ['Disparate Impact', disparate_impact(group_a, group_b, y_pred)]
results.loc[len(results)] = ['Equal Opportunity Difference', equal_opportunity_diff(group_a, group_b, y_pred, y_true)]

results

,Metric,Value
0,Statistical Parity,-0.114024
1,Disparate Impact,0.671597
2,Equal Opportunity Difference,-0.151992


You should get the following

| Metric | Value |
| --- | --- |
| Statistical Parity |-0.11 |
| Disparate Impact | 0.67 |
| Equal Opportunity Difference | -0.15 |


## **Final remarks**
**You should find the same results as in the previous section**.

Note that for each analysis, we only obtained one number for fairness metrics. As mentioned before, in a real life setting, you should calculate a confidence interval by repeating the experiments for different train/test splits. A way to do this is for instance to use a non-parametric bootstrap method as described in section 10.2 from the book [Computer Age Statistical Inference by B.Efron and T.Hastie](https://web.stanford.edu/~hastie/CASI_files/PDF/casi.pdf) that we linked earlier. We give an example of this in the next and last notebook. Note that there will be nothing for you to do in this last notebook.